In [1]:
import pandas
import datetime
import geomag
import numpy

In [2]:
# some constants
cutOffLosVel = 50.

In [3]:
#### In this block we load Velocity data ####
#### In this block we load Velocity data ####
#### In this block we load Velocity data ####
# need the formatted velocity file to read data
outFormattedFile = "/home/bharat/Documents/code/new-vel-data/veldata/saps-vels-20130302.txt"#"../data/new-test-vels-north.txt"
# os.system( 'cat ' + rawVelFileName + ' | tr -s "  " > ' + outFormattedFile )
inpColNames = [ "dateStr", "timeStr", "beam", "range", \
          "geoAzm", "azimCalcMag", "vLos", "spwdth", "pwr", "MLAT", "MLON", "MLT",  \
          "GLAT", "GLON", "radId", "radCode"]
velsDataDF = pandas.read_csv(outFormattedFile, delim_whitespace=True,\
                             header=None, names=inpColNames)
# add a datetime col
velsDataDF["date"] = pandas.to_datetime( \
                        velsDataDF['dateStr'].astype(str) + "-" +\
                        velsDataDF['timeStr'].astype(str), format='%Y%m%d-%H%M')
# for some reason MLAT is a str type, convert it to float
velsDataDF["MLAT"] = velsDataDF["MLAT"].astype(float)
# Also get a normMLT for plotting
velsDataDF['normMLT'] = [x-24 if x >= 12 else x for x in velsDataDF['MLT']]
velsDataDF.head()
#### In this block we load SAPS Velocity data ####
#### In this block we load SAPS Velocity data ####
#### In this block we load SAPS Velocity data ####

,dateStr,timeStr,beam,range,geoAzm,azimCalcMag,vLos,spwdth,pwr,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT
0,20130302,100,0,11,-1.36,-1.5938,18.7037,7.6075,8.2475,54.6651,0.6294,19.9579,43.5040,-75.8121,32,wal,2013-03-02 10:00:00,-4.0421
1,20130302,100,0,12,-1.36,-2.0776,19.7284,7.6075,9.8574,54.9391,0.5341,19.9515,43.7877,-75.8968,32,wal,2013-03-02 10:00:00,-4.0485
2,20130302,100,0,13,-1.36,-2.6106,21.0756,13.6110,10.5196,55.2014,0.4238,19.9442,44.0580,-75.9910,32,wal,2013-03-02 10:00:00,-4.0558
3,20130302,100,0,14,-1.36,-2.9315,21.0756,10.7070,16.0442,55.4872,0.3440,19.9388,44.3553,-76.0649,32,wal,2013-03-02 10:00:00,-4.0612
4,20130302,100,0,15,-1.36,-2.4532,17.0458,9.8640,16.7744,55.8854,0.4085,19.9431,44.7822,-76.0445,32,wal,2013-03-02 10:00:00,-4.0569


In [4]:
# convert azimuth to magn coords
gm = geomag.geomag.GeoMag()
def convert_to_mag_azm(row):
#     print type(row["date"].to_datetime()),row["date"].to_datetime()
    mg = gm.GeoMag(row['GLAT'], \
                   row['GLON'], h=300., time=row['date'].date())
    azm_mag = (round(row['geoAzm'] - mg.dec,2))
    return azm_mag


velsDataDF["magAzm"] = -999.#velsDataDF.apply( convert_to_mag_azm, axis=1 )
velsDataDF.head()

,dateStr,timeStr,beam,range,geoAzm,azimCalcMag,vLos,spwdth,pwr,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT,magAzm
0,20130302,100,0,11,-1.36,-1.5938,18.7037,7.6075,8.2475,54.6651,0.6294,19.9579,43.5040,-75.8121,32,wal,2013-03-02 10:00:00,-4.0421,-999.0
1,20130302,100,0,12,-1.36,-2.0776,19.7284,7.6075,9.8574,54.9391,0.5341,19.9515,43.7877,-75.8968,32,wal,2013-03-02 10:00:00,-4.0485,-999.0
2,20130302,100,0,13,-1.36,-2.6106,21.0756,13.6110,10.5196,55.2014,0.4238,19.9442,44.0580,-75.9910,32,wal,2013-03-02 10:00:00,-4.0558,-999.0
3,20130302,100,0,14,-1.36,-2.9315,21.0756,10.7070,16.0442,55.4872,0.3440,19.9388,44.3553,-76.0649,32,wal,2013-03-02 10:00:00,-4.0612,-999.0
4,20130302,100,0,15,-1.36,-2.4532,17.0458,9.8640,16.7744,55.8854,0.4085,19.9431,44.7822,-76.0445,32,wal,2013-03-02 10:00:00,-4.0569,-999.0


In [5]:
#### In this block we filter for SAPS velocities ####
#### In this block we filter for SAPS velocities ####
#### In this block we filter for SAPS velocities ####
# remove velocies whose magnitude is less than 200 m/s
velsDataDF = velsDataDF[ abs(velsDataDF["vLos"]) >= cutOffLosVel ]
# SAPS(westward) vLos are positive for positive azimuths and vice versa.
# filter the others out
velsDataDF = velsDataDF[ velsDataDF["azimCalcMag"]/velsDataDF["vLos"] > 0. ].reset_index(drop=True)
velsDataDF.head()
#### In this block we filter for SAPS velocities ####
#### In this block we filter for SAPS velocities ####
#### In this block we filter for SAPS velocities ####

,dateStr,timeStr,beam,range,geoAzm,azimCalcMag,vLos,spwdth,pwr,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT,magAzm
0,20130302,100,1,23,1.88,4.1857,51.1515,0.6805,7.7685,58.9197,2.0534,20.0528,48.1145,-75.0895,32,wal,2013-03-02 10:00:00,-3.9472,-999.0
1,20130302,100,2,17,5.12,7.0285,50.8348,12.1012,8.8648,56.5264,2.4084,20.0765,45.6149,-74.6826,32,wal,2013-03-02 10:00:00,-3.9235,-999.0
2,20130302,100,2,18,5.12,7.3964,50.8348,24.8174,8.3442,56.9065,2.5615,20.0867,46.0290,-74.6008,32,wal,2013-03-02 10:00:00,-3.9133,-999.0
3,20130302,100,2,21,5.12,8.3118,51.1515,9.4982,8.3719,58.0392,3.0146,20.1169,47.2614,-74.3609,32,wal,2013-03-02 10:00:00,-3.8831,-999.0
4,20130302,100,2,22,5.12,8.5702,63.0135,12.2166,7.9314,58.4147,3.1647,20.1269,47.6697,-74.2820,32,wal,2013-03-02 10:00:00,-3.8731,-999.0


In [6]:
#### Need prev SAPS data to determine SAPS scatter ####
#### Need prev SAPS data to determine SAPS scatter ####
#### Need prev SAPS data to determine SAPS scatter ####
datFileName = "../data/processedSaps.txt"
sapsDataDF = pandas.read_csv(datFileName, sep=' ', dtype={'dateStr':'str', 'time': 'str'})

sapsDataDF["date"] = pandas.to_datetime( \
                        sapsDataDF['dateStr'] + "-" +\
                        sapsDataDF['time'], format='%Y%m%d-%H%M')
sapsDataDF.head()
#### Need prev SAPS data to determine SAPS scatter ####
#### Need prev SAPS data to determine SAPS scatter ####
#### Need prev SAPS data to determine SAPS scatter ####

,dateStr,sapsLat,sapsMLT,sapsVel,radId,poesLat,poesMLT,dst_date,dst_index,time,date
0,20110107,56.5,17.7543,308.2077,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07
1,20110107,55.5,18.0147,224.1588,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07
2,20110107,56.5,17.8749,307.4328,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07
3,20110107,55.5,18.1324,222.4787,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07
4,20110107,56.5,17.9955,305.4201,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07


In [7]:

#### Choose a time to and get POES boundaries as well ####
#### Choose a time to and get POES boundaries as well ####
#### Choose a time to and get POES boundaries as well ####
# we choose 0840 UT since that time was analyzed in the paper
timeSlctd = datetime.datetime( 2013, 3, 2, 3, 0 )
velAnlysDF = velsDataDF[ velsDataDF["date"] == timeSlctd ].reset_index(drop=True)
sapsSelPrdDF = sapsDataDF[  ( sapsDataDF["date"] - timeSlctd < numpy.timedelta64(40,'m') )\
           & ( sapsDataDF["date"] - timeSlctd > numpy.timedelta64(0,'m') )  ].reset_index(drop=True)
# Now check if there is more than one time period in the selected interval
# ideally there shouldn't be, but if we find one we should do something
# about it (like take an average)...for now its undecided.!
if len( sapsSelPrdDF["time"].unique() ) > 1:
    print "There is more than one time period selected!!!"
    print "NEED TO CHECK SOMETHING WRONG!!!"
    print "NEED TO CHECK SOMETHING WRONG!!!"
    print "NEED TO CHECK SOMETHING WRONG!!!"
poesBndDF = sapsSelPrdDF[ ["poesMLT", "poesLat"] \
                    ].drop_duplicates().reset_index(drop=True)
# Have a normalized MLT for ease of comparison
poesBndDF['normMLT'] = [x-24 if x >= 12 else x for x in poesBndDF['poesMLT']]
poesBndDF.head()
#### Choose a time to and get POES boundaries as well ####
#### Choose a time to and get POES boundaries as well ####
#### Choose a time to and get POES boundaries as well ####

,poesMLT,poesLat,normMLT
0,17.0,69.1861,-7.0
1,18.0,68.8010,-6.0
2,19.0,68.2487,-5.0
3,20.0,67.5669,-4.0
4,21.0,66.8020,-3.0


In [8]:
#### For the chosen time get velocities which fall below POES BND ####
#### For the chosen time get velocities which fall below POES BND ####
#### For the chosen time get velocities which fall below POES BND ####
# Merge POES boundary DF with the vels DF
velAnlysDF["normMLTRound"] = velAnlysDF["normMLT"].astype(int)
print "shape before join--->", velAnlysDF.shape
velAnlysDF = pandas.merge( velAnlysDF, poesBndDF, left_on="normMLTRound", right_on="normMLT", how="inner" )
print "shape after join--->", velAnlysDF.shape
# Filter out velocties above the POES boundary
velAnlysDF = velAnlysDF[ velAnlysDF["MLAT"] < velAnlysDF["poesLat"]\
                       ].reset_index(drop=True).drop_duplicates()
print "shape after filtering by boundary--->", velAnlysDF.shape
#### For the chosen time get velocities which fall below POES BND ####
#### For the chosen time get velocities which fall below POES BND ####
#### For the chosen time get velocities which fall below POES BND ####

shape before join---> (384, 20)
shape after join---> (271, 23)
shape after filtering by boundary---> (271, 23)


In [9]:
# save the data in a csv for lshell fitting
outSelCols = [ "beam", "range", "geoAzm", "azimCalcMag", "magAzm", "vLos"\
           , "MLAT", "MLT", "MLON", "GLAT", "GLON", "radId"\
           , "radCode","normMLT_x", "normMLTRound", "spwdth", "pwr" ]
velAnlysDF.to_csv( "../data/test-losVels.txt", columns=outSelCols, index=False )